# Deep Convolutional Neural Network with TF-Slim
*by Marvin Bertin*
<img src="../images/tensorflow.png" width="400">

In [1]:
import sys  
sys.path.append("../") 

import tensorflow as tf
slim = tf.contrib.slim

%load_ext autoreload
%autoreload 2

## CNN Model with TF-Slim

```
with slim.arg_scope([slim.conv2d, slim.fully_connected],
                    activation_fn=tf.nn.relu,
                    weights_regularizer=slim.l2_regularizer(weight_decay),
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                    biases_initializer=tf.zeros_initializer):
    
    # default parameters for conv2d
    with slim.arg_scope([slim.conv2d], kernel_size = [3,3], padding='SAME'):
        
        # default parameters for max_pool2d    
        with slim.arg_scope([slim.max_pool2d], kernel_size = [2,2], padding = 'SAME'):
        
            # block 1
            net = slim.repeat(inputs, 1, slim.conv2d, 64, scope='conv1')
            net = slim.max_pool2d(net, scope='pool1')

            # block 2
            net = slim.repeat(net, 2, slim.conv2d, 128, scope='conv2')
            net = slim.max_pool2d(net, scope='pool2')

            # block 3
            net = slim.repeat(net, 3, slim.conv2d, 256,, scope='conv3')
            net = slim.max_pool2d(net, scope='pool3')

            # fc1 (use conv2d instead of fully_connected layers)
            net = slim.conv2d(net, 1024, [8, 8], padding='VALID', scope='fc4')
            net = slim.dropout(net, dropout, is_training=is_training, scope='dropout4')

            # fc2
            net = slim.conv2d(net, 1024, [1, 1], scope='fc5')
            net = slim.dropout(net, dropout, is_training=is_training, scope='dropout5')

            # fc3
            net = slim.conv2d(net, output_dim, [1, 1],
                              activation_fn=None,
                              normalizer_fn=None,
                              scope='prediction')
```

## Examine Model Structure

In [3]:
from utils.slim_models import CNNClassifier

image_crop_shape = (64,64,3)
num_class = 5

CNN_model = CNNClassifier("flowers", image_crop_shape , num_class)
CNN_model.examine_model_structure()

Layers
name = CNN_flowers_classifier/conv1/conv1_1/Relu:0             shape = (?, 64, 64, 64)
name = CNN_flowers_classifier/pool1/MaxPool:0                  shape = (?, 32, 32, 64)
name = CNN_flowers_classifier/conv2/conv2_1/Relu:0             shape = (?, 32, 32, 128)
name = CNN_flowers_classifier/conv2/conv2_2/Relu:0             shape = (?, 32, 32, 128)
name = CNN_flowers_classifier/pool2/MaxPool:0                  shape = (?, 16, 16, 128)
name = CNN_flowers_classifier/conv3/conv3_1/Relu:0             shape = (?, 16, 16, 256)
name = CNN_flowers_classifier/conv3/conv3_2/Relu:0             shape = (?, 16, 16, 256)
name = CNN_flowers_classifier/conv3/conv3_3/Relu:0             shape = (?, 16, 16, 256)
name = CNN_flowers_classifier/pool3/MaxPool:0                  shape = (?, 8, 8, 256)
name = CNN_flowers_classifier/fc4/Relu:0                       shape = (?, 1, 1, 1024)
name = CNN_flowers_classifier/fc5/Relu:0                       shape = (?, 1, 1, 1024)
name = CNN_flowers_classifier/p

## Preprocess Images with DatasetProvider
DatasetProvider is located in the *slim_data_provider.py* file

```
def preprocess_image(image, output_height, output_width):
    """Preprocesses the given image.
    Args:
    image: A `Tensor` representing an image of arbitrary size.
    output_height: The height of the image after preprocessing.
    output_width: The width of the image after preprocessing.

    Returns:
    A preprocessed image.
    """
    tf.image_summary('image', tf.expand_dims(image, 0))
    # Transform the image to floats.
    image = tf.to_float(image)

    # Resize and crop if needed.
    resized_image = tf.image.resize_image_with_crop_or_pad(
                    image, output_height, output_width)
    tf.image_summary('resized_image', tf.expand_dims(resized_image, 0))

    # Linearly scales `image` to have zero mean and unit norm.
    return tf.image.image.per_image_standardization(resized_image)

def load_batch_flowers(data_type, output_height, output_width, batch_size):
    dataset = flowers.get_split(data_type, data_dir)
    return load_batch_CNN(output_height, output_width, batch_size)

def load_batch_CNN(dataset, data_type, output_height, output_width, batch_size=32):
    """Loads a single batch of data.

    Args:
      dataset: The dataset to load.
      output_height: The height of the image after preprocessing.
      output_width: The width of the image after preprocessing.
      batch_size: The number of images in the batch.

    Returns:
      images: A Tensor of size [batch_size, height, width, 3],
              image samples that have been preprocessed.
      labels: A Tensor of size [batch_size],whose values range between 0 and num_classes
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32, common_queue_min=8)
    image, label = data_provider.get(['image', 'label'])

    image = preprocess_image(image, output_height, output_width)

    # Batch it up.
    images, labels = tf.train.batch(
        [image, label],
        batch_size=batch_size,
        num_threads=1,  # one queue
        capacity=2 * batch_size)  # two batch per queue max

    return images, labels
```

## Next Lesson
### Deep Convolutional Neural Network for text classification in TensorFlow-Slim
-  Define Deep Convolutional Neural Network for classification task on text dataset.

<img src="../images/divider.png" width="100">